# Obtenir dataset películes

En aquest codi anem a obtenir un dataset de pelicules.
Realitzarem els següents pasos:
- Cargar llibreries.
- Realitzar web scraping sobre la web https://www.themoviedb.org/movie/
- Capturar el llistat de totes les pelicules i obtenir els enllaços
- Accedir a cada pàgina de la pelìcula i agafar les dades
- Tractar aquestes dades i guardar-les en un dataset.
- Visualitzar el dataset per veure si s'ha creat correctament.

Cargar llibreries

In [107]:
from selenium.webdriver.chrome.options import Options 
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import csv
import pandas as pd

Realitzar petició GET sobre la url indicada

In [98]:
url_base = "https://www.themoviedb.org"

options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome("chromedriver.exe", options=options)
driver.get(url_base+"/movie")
#driver.maximize_window()

Obtindre les pelicules del llistat.

Primer fer clic en mostrar más i desprès anar carregant les dades mitjançant l'scroll.

In [99]:
# Fer click en "Mostrar más"
element = driver.find_element_by_link_text("Mostrar más") 
driver.execute_script("arguments[0].click();", element)
time.sleep(1)

# Fer scroll per capturar dades conforme es van carregant
url_single_movies = []
max_iterations = 0
iter=1
while max_iterations < 5:
    scrollHeight = driver.execute_script("return document.documentElement.scrollHeight")
    #print("scrollHeight: "+str(scrollHeight))
    Height=4200*iter
    #print("Height: "+str(Height))
    driver.execute_script("window.scrollTo(0, " + str(Height) + ");")
    if Height > scrollHeight:
        print('End of page')
        break
    time.sleep(random.randint(1, 3))
    max_iterations+=1
    #print("Max iterations: "+str(max_iterations))
    iter+=1
    #print("iter: "+str(iter))

body = driver.execute_script("return document.body")
source = body.get_attribute('innerHTML') 

soup = BeautifulSoup(source, "html.parser")
movies = soup.find_all("div", {"class": "card style_1"})

print(len(movies))

120


Recorrer l'HTML obtingut i capturar els enllaços a la fitxa de cada pel·lícula

In [100]:
# Capturar els enllaços de les págines individuals de les pàgines
for movie in movies:
    title = movie.h2.a["href"] 
    url_single_movies.append(title)

print(url_single_movies)

['/movie/527774', '/movie/587807', '/movie/484718', '/movie/458576', '/movie/464052', '/movie/587996', '/movie/581389', '/movie/793723', '/movie/602269', '/movie/775996', '/movie/649087', '/movie/613504', '/movie/522444', '/movie/630586', '/movie/651571', '/movie/604822', '/movie/590706', '/movie/560144', '/movie/791373', '/movie/755812', '/movie/529203', '/movie/581387', '/movie/508442', '/movie/495764', '/movie/664767', '/movie/399566', '/movie/786300', '/movie/539885', '/movie/577922', '/movie/682254', '/movie/553604', '/movie/599281', '/movie/38700', '/movie/621876', '/movie/520946', '/movie/644092', '/movie/475557', '/movie/531499', '/movie/631939', '/movie/607383', '/movie/299534', '/movie/373571', '/movie/621013', '/movie/299536', '/movie/666750', '/movie/582306', '/movie/615677', '/movie/737568', '/movie/384018', '/movie/545609', '/movie/400160', '/movie/338762', '/movie/741228', '/movie/714277', '/movie/765169', '/movie/512200', '/movie/283566', '/movie/531219', '/movie/461130

Creem algunes funcions auxiliars que ens ajudaran a tractar les dades de la fitxa

In [73]:
# Get directors    

# Get link img cartell

# Get Titol i Idioma original

# Get año de estreno

# Get Puntiación

Obrim el fitxer csv on guardarem les dades obtingudes i escrivim la primera línea que sera el nom de les columnes

In [120]:
data_file = open('dataset_peliculas.csv', 'w', newline='', encoding='utf-8') 
csv_writer = csv.writer(data_file, delimiter=";") 
csv_writer.writerow(["Título","Título original","Idioma original","Duración","Directores","Sinopsis","Año de estreno","Clasificación por edad","Imagen cartel","Puntuación"]) 

132

Accedim a la fitxa de cada pel·lícula i anem capturant les dades en format JSON per despres guardar-les més facilment en un archiu csv

In [121]:
for url_movie in url_single_movies:
    driver.get(url_base+url_movie)
    body = driver.execute_script("return document.body")
    source = body.get_attribute('innerHTML') 
    soup = BeautifulSoup(source, "html.parser")

    poster_wrapper = soup.find("div", {"class": "poster_wrapper"})
    header_wrapper = soup.find("div", {"class": "header_poster_wrapper"})
    column_wrapper = soup.find("div", {"class": "column "})

    movie =  [
        header_wrapper.find("h2").a.contents[0], # Título
        "Título original", # Título original
        "Idioma original", # Idioma original
        "Duración", # Duración
        "Directores", # Directores
        "Sinopsis", # Sinopsis
        header_wrapper.find("span", {"class": "release"}).contents[0].strip(), # Año de estreno
        "Clasificación por edad", # Clasificación por edad
        "Imagen cartel", # Imagen cartel
        "Puntuación" # Puntuación 
    ]

    csv_writer.writerow(movie)
    time.sleep(random.randint(1, 3))

Tanquen el driver de selenium i el archiu csv

In [122]:
data_file.close() 

Tanquem el driver de selenium

In [ ]:
driver.close()

Per comprovar que s'han guardat correctament, carreguem el csv en una variable i vegem els primers registres

In [126]:
df_movies = pd.read_csv("dataset_peliculas.csv", sep=";", encoding="utf-8")
df_movies.head(10)

,Título,Título original,Idioma original,Duración,Directores,Sinopsis,Año de estreno,Clasificación por edad,Imagen cartel,Puntuación
0,Raya y el último dragón,Título original,Idioma original,Duración,Directores,Sinopsis,5/3/2021 (ES),Clasificación por edad,Imagen cartel,Puntuación
1,Tom y Jerry,Título original,Idioma original,Duración,Directores,Sinopsis,26/2/2021 (US),Clasificación por edad,Imagen cartel,Puntuación
2,El rey de Zamunda,Título original,Idioma original,Duración,Directores,Sinopsis,5/3/2021 (ES),Clasificación por edad,Imagen cartel,Puntuación
3,Monster Hunter,Título original,Idioma original,Duración,Directores,Sinopsis,26/3/2021 (ES),Clasificación por edad,Imagen cartel,Puntuación
4,Wonder Woman 1984,Título original,Idioma original,Duración,Directores,Sinopsis,18/12/2020 (ES),Clasificación por edad,Imagen cartel,Puntuación
5,Bajocero,Título original,Idioma original,Duración,Directores,Sinopsis,29/1/2021 (ES),Clasificación por edad,Imagen cartel,Puntuación
6,Barrenderos espaciales,Título original,Idioma original,Duración,Directores,Sinopsis,5/2/2021 (US),Clasificación por edad,Imagen cartel,Puntuación
7,Centinela,Título original,Idioma original,Duración,Directores,Sinopsis,5/3/2021 (US),Clasificación por edad,Imagen cartel,Puntuación
8,Pequeños detalles,Título original,Idioma original,Duración,Directores,Sinopsis,26/2/2021 (ES),Clasificación por edad,Imagen cartel,Puntuación
9,A descubierto,Título original,Idioma original,Duración,Directores,Sinopsis,15/1/2021 (ES),Clasificación por edad,Imagen cartel,Puntuación
